

# Mesh generation

This example shows how to generate a mesh from a CAD model. The CAD model is
imported from a file, and the mesh is generated using the Ansys PRIME API.


In [ ]:
import os
from pathlib import Path

from ansys.meshing import prime
from ansys.meshing.prime.graphics import Graphics

## Parameters for the script
The following parameters are used to control the script execution. You can
modify these parameters to suit your needs.




In [ ]:
GRAPHICS_BOOL = False  # Set to True to display the graphics
OUTPUT_DIR = Path(Path(__file__).parent, "outputs")  # Output directory

## Start a PRIME session
Start a PRIME session and get the model from the client.




In [ ]:
prime_client = prime.launch_prime(timeout=120)
print(prime_client)

# Get the model from the client
model = prime_client.model

## Load the CAD file
Load the CAD file from the previous example. The file is loaded into the
model, and the part is extracted. The part is then summarized to get the
details of the part.



In [ ]:
# Load the FMD file
modeling_file = Path(OUTPUT_DIR, "modeling_demo.fmd")
file_io = prime.FileIO(model)
file_io.import_cad(
    file_name=modeling_file,
    params=prime.ImportCadParams(
        model=model,
    ),
)

# Review the part
part = model.get_part_by_name("modelingdemo")
part_summary_res = part.get_summary(prime.PartSummaryParams(model, print_mesh=False))
print(part_summary_res)

if GRAPHICS_BOOL:
    display = Graphics(model=model)
    display()

## Mesh generation
The mesh is generated using the Ansys PRIME API. The mesh is generated using
the following steps:

1. Initialize the connection tolerance and other parameters.
2. Scaffold the part.
3. Mesh the surfaces.
4. Write the mesh to a file.




In [ ]:
# Initialize the connection tolerance and other parameters --
#
# Target element size
element_size = 0.5

# Initialize the parameters
params = prime.ScaffolderParams(
    model,
    absolute_dist_tol=0.1 * element_size,
    intersection_control_mask=prime.IntersectionMask.FACEFACEANDEDGEEDGE,
    constant_mesh_size=element_size,
)

# Get existing topoface or topoedge IDs
faces = part.get_topo_faces()
beams = []

scaffold_res = prime.Scaffolder(model, part.id).scaffold_topo_faces_and_beams(
    topo_faces=faces, topo_beams=beams, params=params
)
print(scaffold_res)

## Surface meshing
The surface mesh is generated using the previous element size and the
topological faces of the part.




In [ ]:
surfer_params = prime.SurferParams(
    model=model,
    size_field_type=prime.SizeFieldType.CONSTANT,
    constant_size=element_size,
    generate_quads=True,
)

surfer_result = prime.Surfer(model).mesh_topo_faces(part.id, topo_faces=faces, params=surfer_params)

# Display the mesh
if GRAPHICS_BOOL:
    display = Graphics(model=model)
    display()

## Export the mesh
The mesh is exported to a CDB file. The CDB file can be used to create a
MAPDL case.




In [ ]:
mapdl_cdb = Path(OUTPUT_DIR, "modeling_demo.cdb")
file_io.export_mapdl_cdb(mapdl_cdb, params=prime.ExportMapdlCdbParams(model))
assert os.path.exists(mapdl_cdb)
print(f"MAPDL case exported at {mapdl_cdb}")

## Close the PRIME session
Close the PRIME session to release the resources. This is important to
prevent memory leaks.



In [ ]:
prime_client.exit()